In [1]:
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code.envs.MountainCar import MultiMountainCar, LookupPolicy
from wip.Code.train import make_dataset_simple
import time
from collections import OrderedDict

In [2]:

BATCH_SIZE = 64#512
SIM_TIME = 1
MAX_ITER = 200
USE_JIT = True

device = torch.device('cpu')

env = MultiMountainCar(device)

#torch.backends.cudnn.enabled = False


In [3]:
from Code.Networks import Selector, DynNetwork, OuterWrapper, LSTMWrapper, ReLuWrapper, DummyNeuron, make_SequenceWrapper
from Code.NewNeurons import SeqOnlySpike, CooldownNeuron

base_config = {
    'ALPHA': 0,
    'BETA': 0,
    'OFFSET': 2,
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

heavyside = {
    **base_config,
    'BETA': 1,
}

mem_loop = OrderedDict([
    ('input', 1),
    ('pre_mem', [['input', 'output'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['pre_mem'], CooldownNeuron(128, heavyside), nn.Linear]),
])

architecture = OrderedDict([
    ('input', 1),
    ('mem_loop', [['input'], make_SequenceWrapper(DynNetwork(mem_loop), USE_JIT), None]),
    ('post_mem', [['input', 'mem_loop'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['post_mem'], DummyNeuron(1), nn.Linear]),
])

architecturelstm = OrderedDict([
    ('input', 3),
    ('obs', [['input'], Selector(0, 2), None]),
    ('probe', [['input'], Selector(2, 1), None]),
    ('lstm', [['obs'], LSTMWrapper(2, 128), None]),
    ('post_mem', [['probe', 'lstm'], ReLuWrapper(128), nn.Linear]),
    ('output', [['post_mem'], DummyNeuron(2), nn.Linear]),
])

#TODO: fix output


In [4]:
#144, 150, 137, 150

model = OuterWrapper(DynNetwork(architecture), device, USE_JIT)

#model = (OuterWrapper(DynNetwork(architecturelstm), device, True))



In [5]:
import gym
testenv = gym.make('MountainCar-v0')

def validate(num_runs, render=False):
    sum = 0
    for i in range(num_runs):
        obs = testenv.reset()
        state = None
        for t in range(300):
            if render:
                testenv.render()
            output, state = model(torch.tensor([[[obs[0]]]], dtype=torch.float), state)
            action = 2 if output > 0 else 0
            obs, _, done, _ = testenv.step(action)
            if done:
                #print(t+1)
                sum += t + 1
                break
    print('Validation: ', sum/num_runs)


/opt/conda/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
teacher = LookupPolicy(device)
bce = nn.BCELoss(reduction='none') #reduction='sum'
optimizer = optim.Adam(model.parameters(), lr=5e-4)#0.00001


In [7]:
#optimizer = optim.Adam(model.parameters(), lr=1e-3)#0.00006

for i in range(5000):
    model.zero_grad()
    observation = env.reset(BATCH_SIZE)
    state = None
    loss = 0
    div = 0
    for t in range(MAX_ITER):
        output, state = model(observation[:,:1].unsqueeze(0), state)
        target = teacher(observation)/2
        #print(observation[:,:1].unsqueeze(0).shape, output.shape, target.shape)
        action = (output.squeeze() > 0) * 2.0
        observation, _, done, _ = env.step(action)
        loss = loss + (bce(torch.sigmoid(output.squeeze()), target) * (~done).float()).sum()
        div = div + (~done).float().sum()
        #print(t, loss)
        if done.all():
            break
    loss = loss / div
    if i%10 == 0:
        print(loss.item(), t+1, i) #, ((outputs>0.5) != targets).sum()
    loss.backward()
    optimizer.step()
    if i%100 == 0:
        validate(10)
    

0.8053970336914062 200 0
Validation:  200.0
0.6891964673995972 200 10
0.6691575050354004 200 20
0.6340159773826599 200 30
0.6083673238754272 200 40
0.5603190064430237 200 50
0.5072405934333801 200 60
0.4292979836463928 200 70
0.44322651624679565 200 80
0.493443101644516 200 90
0.431893914937973 200 100
Validation:  200.0
0.2972724735736847 110 110
1.6417969465255737 200 120
1.0377873182296753 200 130
0.6810043454170227 200 140
0.606848955154419 200 150
0.5674368143081665 200 160
0.47181835770606995 200 170
0.38894858956336975 108 180
0.33252933621406555 136 190
0.22644995152950287 107 200
Validation:  102.2
0.18850599229335785 106 210
0.19647754728794098 107 220
0.19306433200836182 107 230
0.15543511509895325 104 240
0.16043783724308014 105 250
0.17803466320037842 105 260
0.5685468912124634 200 270
2.3272669315338135 200 280
0.8131563067436218 200 290
0.7446719408035278 200 300
Validation:  200.0
0.6820564270019531 200 310
0.6378422379493713 200 320
0.7526577711105347 200 330
0.5318964

RuntimeError: KeyboardInterrupt: 

At:
  /opt/conda/lib/python3.6/site-packages/torch/tensor.py(30): wrapped
  ../Code/NewNeurons.py(40): forward
  /opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py(532): __call__
  ../Code/Networks.py(144): forward
  /opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py(532): __call__
  <ipython-input-7-24e9036caa59>(10): <module>
  /opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py(3319): run_code
  /opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py(3242): run_ast_nodes
  /opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py(3051): run_cell_async
  /opt/conda/lib/python3.6/site-packages/IPython/core/async_helpers.py(68): _pseudo_sync_runner
  /opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py(2874): _run_cell
  /opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py(2848): run_cell
  /opt/conda/lib/python3.6/site-packages/ipyker

In [7]:
mydict = {}

def train_batch(inputs, targets, mask):
    model.zero_grad()
    outputs, _ = model(inputs)
    loss = (bce(torch.sigmoid(outputs), targets) * mask).sum() / mask.sum()
    loss.backward()
    optimizer.step()
    return loss.item()


def train_dataset(num_batches, num_epochs):
    obs, target, mask = make_dataset_simple(num_batches, BATCH_SIZE, MAX_ITER, model, teacher, device, env)
    for e in range(num_epochs):
        start = time.time()
        idc = torch.randperm(obs.shape[1], device=device)
        for i in range(num_batches):
            base = i*BATCH_SIZE
            batch_obs = obs[:, idc[base:base + BATCH_SIZE]]
            batch_targets = target[:, idc[base:base + BATCH_SIZE]]
            batch_mask = mask[:, idc[base:base + BATCH_SIZE]]
            #print(batch_obs.shape, batch_targets.shape, batch_mask.shape)
            loss = train_batch(batch_obs, batch_targets, batch_mask)
        for p in model.parameters():
            if torch.isnan(p).any():
                raise Exception('Corrupted Model')
        print(loss, time.time()-start)
            #if i%10 == 0:
                #print(loss.item(), (loss/targets.view(-1).var()).item(), i)



In [ ]:
for i in range(10):
    print('Bigstep: ', i)
    train_dataset(100, 10)
    validate(10)

Bigstep:  0
0.09936636686325073 63.01702380180359
0.09963464736938477 61.76665282249451
0.08759665489196777 61.996296882629395
0.11712847650051117 58.70791983604431
0.10174913704395294 55.109790563583374
0.0598505400121212 54.995145320892334
0.073218934237957 55.20386624336243
0.07360125333070755 55.19120740890503
0.1353931725025177 55.456132888793945
0.09279230237007141 55.06258201599121
Validation:  138.2
Bigstep:  1
0.11334297060966492 54.9138605594635
0.0551205649971962 54.91706037521362
0.048263706266880035 55.11578369140625
0.06823704391717911 62.34108829498291
0.05546610802412033 62.355822801589966
0.06327603757381439 62.11244034767151
0.043474551290273666 61.89683151245117
0.08305107057094574 61.76122975349426
0.06539105623960495 61.2263822555542
0.05194448307156563 61.78586673736572
Validation:  131.5
Bigstep:  2
0.06580055505037308 61.19282054901123
0.04305011406540871 66.03960156440735
0.04320993274450302 56.65983605384827
0.03785112500190735 62.45754671096802
0.028821770101

In [11]:
validate(1, render=True)

Validation:  105.0


In [14]:
obs = testenv.reset()
state = None
for t in range(300):
    output, state = model(torch.tensor([[[obs[0]]]], dtype=torch.float), state)
    action = 2 if output > 0 else 0
    obs, _, done, _ = testenv.step(action)
    testenv.render()
    #print(output)


In [ ]:

testenv.close()